In [1]:
import time
import math

In [2]:
def round(primes):
  zero = 0
  one = 0
  two = 0
  three = 0
  four = 0
  five = 0
  six = 0
  seven = 0
  eight = 0
  nine = 0
  ten = 0
  eleven = 0
  twelve = 0
  thirteen = 0
  fourteen = 0
  fifteen = 0
  sixteen = 0
  seventeen = 0
  eighteen = 0
  nineteen =0
  twenty = 0
  other = 0
  for i in primes:
    if i < 0.1:
      zero+=1
    elif i > 0.1 and i<0.2:
      one += 1
    elif i > 0.2 and i<0.3:
      two += 1
    elif i > 0.3 and i<0.4:
      three += 1
    elif i > 0.4 and i<0.5:
      four +=1
    elif i > 0.5 and i<0.6:
      five +=1
    elif i > 0.6 and i<0.7:
      six += 1
    elif i > 0.7 and i<0.8:
      seven += 1
    elif i > 0.8 and i<0.9:
      eight += 1
    elif i > 0.9 and i<1:
      nine += 1
    elif i > 1 and i<1.1:
      ten += 1
    elif i > 1.1 and i<1.2:
      eleven += 1
    elif i > 1.2 and i<1.3:
      twelve += 1
    elif i > 1.3 and i<1.4:
      thirteen += 1
    elif i > 1.4 and i<1.5:
      fourteen += 1
    elif i > 1.5 and i<1.6:
      fifteen += 1
    elif i > 1.6 and i<1.7:
      sixteen += 1
    elif i > 1.7 and i<1.8:
      seventeen += 1
    elif i > 1.8 and i<1.9:
      eighteen += 1
    elif i > 1.9 and i<2.0:
      nineteen += 1
    elif i > 2.0 and i< 2.1:
      twenty += 1
    else:
      other+=1
  print("0.0: ", zero)
  print("0.1: ", one)
  print("0.2: ", two)
  print("0.3: ", three)
  print("0.4: ", four)
  print("0.5: ", five)
  print("0.6: ", six)
  print("0.7: ", seven)
  print("0.8: ", eight)
  print("0.9: ", nine)
  print("1.0: ", ten)
  print("1.1: ", eleven)
  print("1.2: ", twelve)
  print("1.3: ", thirteen)
  print("1.4: ", fourteen)
  print("1.5: ", fifteen)
  print("1.6: ", sixteen)
  print("1.7: ", seventeen)
  print("1.8: ", eighteen)
  print("1.9: ", nineteen)
  print("2.0: ", twenty)
  print("other: ", other)

In [3]:
def isprime(n) :
    if (n <= 1):
        return False
    if (n <= 3):
        return True
 
    # This is checked so that we can skip
    # middle five numbers in below loop
    if (n % 2 == 0 or n % 3 == 0):
        return False
 
    i = 5
    while(i * i <= n):
        if (n % i) == 0 or n % (i + 2) == 0:
            return False
        i = i + 6
 
    return True

In [4]:
def gaussian_primes():#it fnds all primes numbers of the type a^2+b^2 (Guassian integers) < 10,000
  _list = list()
  for a in range(1, 10):
      for b in range(1,a):
         q = a**2 + b**2 #norm of the guassian integer (a+bi)
         if q < 100 and isprime(q):
            _list.append(math.atan(b/a))
  return _list


def sqrt_two_primes():#it fnds all primes numbers of the type a^2-2b^2
  _list = list()
  for a in range(1, 1200):
      for b in range(1,800):
        if b < 2*a/ 3:
          q = a**2 - 2*(b**2)
          if q < 100000 and isprime(q):
            _list.append(math.atanh(b*math.sqrt(2)/a))
  return _list



def sqrt_five_primes():#it fnds all primes numbers of the type a^2-5b^2
  _list = list()
  for a in range(1, 3000):
      for b in range(1,1000):
        if b < a/ 3:
          q = a**2 - 5*(b**2)
          if q < 400000 and isprime(q):
            _list.append(math.atanh(b*math.sqrt(5)/a))
  return _list

def sqrt_three_primes(x):#it fnds all primes numbers of the type a^2-3b^2
  _list = list()
  for a in range(1, 3000):
      for b in range(1,1500):
        if b < a/3: #we are actualy finding the a/sqrt(3)
          q = a**2 - 3*(b**2)
          if q < x and isprime(q):
            _list.append(math.atanh(b*math.sqrt(3)/a))

  return _list

'''
def qdrt_eleven_primes(k):#it fnds all primes numbers of the type a^4+11b^4
  _list = list()
  for a in range(1, 200):
      for b in range(1,200):
        if b < a: #we are actualy finding the a/sqrt(3)
          q = a**4 + 11*(b**4)
          if q < k and isprime(q):
            _list.append(b/a)

  return _list
'''


'\ndef qdrt_eleven_primes(k):#it fnds all primes numbers of the type a^4+11b^4\n  _list = list()\n  for a in range(1, 200):\n      for b in range(1,200):\n        if b < a: #we are actualy finding the a/sqrt(3)\n          q = a**4 + 11*(b**4)\n          if q < k and isprime(q):\n            _list.append(b/a)\n\n  return _list\n'

In [5]:
def qdrt_eleven_primes(m):#it fnds all primes numbers of the type a^2-3b^2
  _list = list()
  for a in range(1, 800):
      for b in range(1,800):
        if b < a: #we are actualy finding the a/sqrt(3)
          q = a**4 + 11*(b**4)
          if q < m and isprime(q):
            _list.append(b/a)

  return _list

def f(t):
    return 1 + (11 * t **4)
def r(t):
    return 1/ math.sqrt(f(t))
'''
def d(k,n):
    return r((2*k+1)/(2*n))
def D(t,n):
    c = math.ceil(n*t)
    x = 0
    for k in range(0,c):
        x+= d(k,n)/n
    return x
''''
delta_x = 0.01

def d(n):
    return delta_x * r(n*delta_x)

'''
def x1
    for i in range(0,100):
        if f > 0.01:
            return i
    '''        
        

def distribution(m,n):
    #depends on n from the the function D(t, n) and m from the function qdrt_eleven_primes
    #t is the element from the list return from qdrt_eleven_primes
    _list = []
    qdrt = qdrt_eleven_primes(m)
    for t in qdrt:
        _list.append(D(t,n))
    return _list
    

SyntaxError: EOL while scanning string literal (<ipython-input-5-5d705e5b0af6>, line 25)

n = 100
m = 625 * (10**8)
y = distribution(m,n)
round(y)

n = 100
m = 800 ** 4
y = distribution(m,n)
round(y)
#the maximum number

Delta_x=0.01
D=0
For n in range(1,100)
   D==D+Delta_x*r(n*Delta_x)
   If D>0.1
      i_1==n
      Append i_1
      Break
For n in range(1+i_1,100)
   D==D+Delta_x*r(n*Delta_x)
   If D>0.2
      i_2==n
      Append i_2
      Break
For n in range (1+i_2,100)

(Consider i_1,i_2, up to i_10)

For n in range(1+i_9,100)
   If D==D+Delta_x*r(n*Delta_x)
      i_10==n
Append i_10

(Now the list D_inverse will contain the element i_1, i_2,...i_9,i_10.)

N=100
One=0
Two=0
...
Ten=0
For a in range(1,N)
   For b in range(1,a)
     q=a**4 + 11*b**4
     M=N^4
     If q<M
        If q is a prime
            If b/a<i_1 then one == one+1
            Else if b/a<i_2 then two==two+1
            Else if b/a < i_3 then three == three+1
            ...
            Else if b/a<I_10 then ten == ten+1
Print( one, two , three,...)

def qdrt_eleven_primes(m):#it fnds all primes numbers of the type a^4+11b^4
  _list = list()
  for a in range(1, 800):
      for b in range(1,800):
        if b < a: #we are actualy finding the a/sqrt(3)
          q = a**4 + 11*(b**4)
          if q < m and isprime(q):
            _list.append(b/a)

  return _list

def f(t):
    return 1 + 11 * t **4
def r(t):
    return 1/ math.sqrt(f(t))

Delta_x=0.01
D=0
i_list = []
for n in range(1,100):
    D = D+Delta_x*r(n*Delta_x)
    if D>0.1:
        i_1 = n
        i_list.append(D)
        break
for n in range(1+i_1,100):
    D = D + Delta_x*r(n*Delta_x)
    if D>0.2:
        i_2 = n
        i_list.append(D)
        break
        
for n in range(1+i_2,100):
    D = D + Delta_x*r(n*Delta_x)
    if D>0.3:
        i_3 = n
        i_list.append(D)
        break

for n in range(1+i_3,100):
    D = D + Delta_x*r(n*Delta_x)
    if D>0.4:
        i_4 = n
        i_list.append(D)
        break
for n in range(1+i_4,100):
    D = D + Delta_x*r(n*Delta_x)
    if D>0.5:
        i_5 = n
        i_list.append(D)
        break
        
for n in range(1+i_5,100):
    D = D + Delta_x*r(n*Delta_x)
    if D>0.6:
        i_6 = n
        i_list.append(D)
        break
for n in range(1+i_6,100):
    D = D + Delta_x*r(n*Delta_x)
    if D>0.7:
        i_7 = n
        i_list.append(D)
        break
for n in range(1+i_7,100):
    D = D + Delta_x*r(n*Delta_x)
    if D>0.8:
        i_8 = n
        i_list.append(D)
        break
    else:
        i_8 = i_7
for n in range(1+i_8,100):
    D = D + Delta_x*r(n*Delta_x)
    if D>0.9:
        i_9 = n
        i_list.append(D)
        break
    else:
        i_9 = i_8
for n in range(1+i_9,100):
    D = D + Delta_x*r(n*Delta_x)
    if D>1:
        i_10 = n
        i_list.append(D)
        break
    else:
        i_10= i_9


In [5]:
#i_lst


NameError: name 'i_lst' is not defined

In [6]:

N = 100
one=0
two=0
three = 0
four = 0
five = 0
six  = 0
seven = 0
for a in range(1,N):
    for b in range(1,a):
        q = a**4 + 11*b**4
        M = (N**4)/12
        if q<M and isprime(q):
            if b/a <= test_list[0]/10000:
                one+=1
            elif b/a <=test_list[1]/10000:
                two+=1
            elif b/a <= test_list[2]/10000:
                three+=1
            elif b/a <= test_list[3]/10000:
                four += 1
            elif b/a <= test_list[4]/10000:
                five+=1
            elif b/a <= test_list[5]/10000:
                six += 1
            elif b/a <= test_list[6]/10000:
                seven += 1
           
print( one, two , three, four, five, six, seven)

NameError: name 'test_list' is not defined

In [ ]:
def R(t):
    return 1/ math.sqrt(1 + 11 * t **4)
def R (t, D, n):
    1/nrt(1+D*t**n,n/2)

delta_x=0.0001
d=0
i_lst = []
counter = 0
limit = 0.1
i_ = 1
while counter <11:
    for j in range(i_,10000):
        d = d+delta_x*R(j*delta_x)
        if d>limit:
            i_ = j
            i_lst.append(j)
            break
    limit+=0.1
    counter+=1

# Function that find the nth root of a number

In [6]:
#the function is used to find the nth root of a non-negative number
def nrt(numb,e=2):
    """numb should >0
    e represent the nth root , 2=square root , 3=cube root, e= for only positive numbeer"""
    guess = numb * 1/e
    avg = ((e-1)*guess + (numb/(guess**(e-1))))/e
    while avg!= guess:
        guess = avg
        avg = ((e-1)*guess + (numb/(guess**(e-1))))/e #Newsthon's method formula
    return guess                 



# Inverse of a Distribution for Primes of the Form a^n + D*b^n

In [7]:
def inverse_distribution_primes_a_b_nth(D, n):
    delta_x=0.0001
    integral = 0
    i_lst = []
    counter = 0
    limit = 0.1
    i_ = 1
    while counter <11:
        for j in range(i_,10000):
            integral += delta_x*(1/ (nrt(1 + D * (j*delta_x) **n,n)**2))
            if integral>limit:
                i_ = j
                i_lst.append(j)
                break
        limit+=0.1
        counter+=1
    return i_lst

# Distribution of prime numbers of the form a^n + D*b^n

In [8]:
def list_creator(n):
    List = []
    for i in range(n):
        List.append(0)
    return List

In [9]:
def distribution_primes_a_b(N, D, n, LIST):
    len_List = len(LIST)
    counter_list = list_creator(len_List)
    for a in range(1,N):
        for b in range(1,a):
            q = a**n + D*b**n
            M = (N**n)/(D+1)
            if q<M and isprime(q):
                for i in range(len(LIST)):
                    if i==0 and b/a <= LIST[0]/10000:
                        counter_list[0]+=1 
                    elif i!=0 and b/a <= LIST[i]/10000 and b/a > LIST[i-1]/10000:
                        counter_list[i]+=1 
    return counter_list
    

In [10]:
def deviation(dist_list):
    return max(dist_list)/min(dist_list)

In [11]:
def testing(rangeN, D, n, inverse_list):
    print(f"Distribution of primes of the form a^{n} + {D}b^{n}")
    test_dic = dict()
    for N in range(100,rangeN+100, 100):
        distribution = distribution_primes_a_b(N, D, n, inverse_list)
        #test_dic[distribution] = deviation(distribution)
        test_dic.update({f"N = {N}  "+str(distribution): deviation(distribution)})
        
    return test_dic

In [12]:
s = dict()
s["zouby"] = 2

In [13]:
s

{'zouby': 2}

# Testing: Inverse Distruntion for primes a^4 + 11b^4

In [14]:
eleven_fourth = inverse_distribution_primes_a_b_nth(11, 4)
eleven_fourth

[1001, 2003, 3025, 4113, 5360, 6963, 9359]

In [15]:
dist_eleven_fourth = distribution_primes_a_b(100,11,4, eleven_fourth) # def distribution_primes_a_b(N, D, n, LIST):
dist_eleven_fourth

[7, 6, 11, 8, 11, 12, 10]

In [16]:
x = testing(1000, 11, 4, eleven_fourth)
x

Distribution of primes of the form a^4 + 11b^4


{'N = 100  [7, 6, 11, 8, 11, 12, 10]': 2.0,
 'N = 200  [28, 32, 31, 35, 37, 36, 34]': 1.3214285714285714,
 'N = 300  [62, 68, 63, 67, 84, 73, 77]': 1.3548387096774193,
 'N = 400  [114, 117, 106, 122, 123, 125, 119]': 1.179245283018868,
 'N = 500  [171, 166, 158, 179, 178, 195, 177]': 1.2341772151898733,
 'N = 600  [237, 249, 228, 250, 238, 262, 258]': 1.1491228070175439,
 'N = 700  [313, 320, 322, 324, 320, 340, 335]': 1.0862619808306708,
 'N = 800  [388, 396, 409, 408, 416, 430, 421]': 1.1082474226804124,
 'N = 900  [485, 501, 519, 504, 511, 522, 531]': 1.0948453608247424,
 'N = 1000  [593, 615, 623, 610, 620, 634, 649]': 1.0944350758853287}

# Testing: Inverse Distruntion for primes a^4 + 2b^4

In [17]:
two_fourth = inverse_distribution_primes_a_b_nth(2, 4)
two_fourth

[1001, 2000, 3003, 4018, 5059, 6154, 7344, 8695]

In [18]:
two = testing(1000, 2, 4, two_fourth)
two

Distribution of primes of the form a^4 + 2b^4


{'N = 100  [17, 18, 25, 18, 17, 22, 17, 26]': 1.5294117647058822,
 'N = 200  [70, 66, 73, 64, 70, 68, 57, 76]': 1.3333333333333333,
 'N = 300  [140, 136, 143, 144, 154, 133, 126, 138]': 1.2222222222222223,
 'N = 400  [224, 228, 240, 251, 242, 228, 215, 234]': 1.1674418604651162,
 'N = 500  [331, 336, 362, 376, 355, 348, 344, 361]': 1.13595166163142,
 'N = 600  [459, 468, 504, 521, 494, 479, 494, 498]': 1.1350762527233116,
 'N = 700  [621, 631, 651, 670, 670, 632, 665, 669]': 1.0789049919484701,
 'N = 800  [802, 825, 817, 839, 865, 797, 830, 862]': 1.0853199498117942,
 'N = 900  [1016, 1032, 1031, 1042, 1063, 984, 1025, 1065]': 1.0823170731707317,
 'N = 1000  [1227, 1256, 1273, 1266, 1282, 1198, 1258, 1286]': 1.0734557595993321}

# Testing: Inverse Distruntion for primes a^4 + 3b^4

In [19]:
three_fourth = inverse_distribution_primes_a_b_nth(3, 4)
three_fourth

[1001, 2000, 3006, 4028, 5091, 6236, 7529, 9085]

In [22]:
three = testing(1000, 3, 4, three_fourth)
three

Distribution of primes of the form a^4 + 3b^4


{'N = 100  [19, 27, 20, 20, 24, 18, 21, 27]': 1.5,
 'N = 200  [67, 70, 80, 72, 74, 69, 78, 82]': 1.2238805970149254,
 'N = 300  [152, 149, 166, 149, 151, 154, 153, 153]': 1.1140939597315436,
 'N = 400  [250, 242, 271, 263, 238, 247, 254, 252]': 1.138655462184874,
 'N = 500  [372, 376, 392, 393, 354, 371, 382, 376]': 1.1101694915254237,
 'N = 600  [510, 526, 540, 517, 500, 511, 532, 528]': 1.08,
 'N = 700  [688, 688, 708, 675, 675, 669, 718, 685]': 1.0732436472346787,
 'N = 800  [868, 871, 901, 887, 866, 868, 893, 880]': 1.0404157043879907,
 'N = 900  [1063, 1061, 1114, 1086, 1076, 1078, 1113, 1088]': 1.0499528746465598,
 'N = 1000  [1292, 1301, 1331, 1328, 1321, 1272, 1344, 1295]': 1.0566037735849056}

# Testing: Inverse Distruntion for primes a^3 + 2b^3

In [ ]:
two_third = inverse_distribution_primes_a_b_nth(2, 3)
two_third

In [ ]:
two_3 = testing(500, 2, 3, two_third)
two_3

# Testing: Inverse Distruntion for primes a^3 + 3b^3

In [ ]:
three_third = inverse_distribution_primes_a_b_nth(3, 3)
three_third

In [ ]:
three_3 = testing(500, 3, 3, three_third)
three_3